<a href="https://colab.research.google.com/github/Hanifdia/IORS_Lecture/blob/main/kNN/kNN_Teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from osgeo import gdal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy.spatial import distance


# gdal constants
from gdalconst import *


# inform to use GDAL exceptions
gdal.UseExceptions()

In [ ]:
# Import image dataset
fname_img = "Test_Data.tif"
! gdown --id 1DsBMWM-njnfZ7ac64Gwq50-NnExyACAE

In [56]:
# open dataset
dataset = gdal.Open(fname_img, GA_ReadOnly)

In [57]:
# retrieve metadata from raster
rows = dataset.RasterYSize
columns = dataset.RasterXSize
N = rows * columns
bands = dataset.RasterCount

# print basic metadata
print ("image metadata:")
print (rows, "rows x", columns, "columns x", bands, "bands")

image metadata:
769 rows x 1147 columns x 4 bands


In [58]:
# parameters for kNN algorithm
k = 5

In [60]:
# retrieve arrays from input image, 3 channels in this case
array_R = dataset.GetRasterBand(1).ReadAsArray().astype(float)
array_G = dataset.GetRasterBand(2).ReadAsArray().astype(float)
array_B = dataset.GetRasterBand(3).ReadAsArray().astype(float)
array_RGB = np.zeros((rows, columns, bands), dtype=np.uint8)
array_RGB[:,:,0] = array_R
array_RGB[:,:,1] = array_G
array_RGB[:,:,2] = array_B
array_RGB_copy = array_RGB.copy()

In [ ]:
# Import Training data (Label Data)

! gdown --id 1cxs1vwnhRbV5u7e6dLJATJnY8q6o2_Gu
samples = pd.read_csv("Samples.csv").values.tolist()

array_R_flatten = array_R.flatten()
array_G_flatten = array_G.flatten()
array_B_flatten = array_B.flatten()



In [53]:
# kNN Classification, sorting data, and category assignment base on k nearest neighbour
classification = []
for pixel in range(len(array_R_flatten)):
  r = array_R_flatten[pixel]
  g = array_G_flatten[pixel]
  b = array_B_flatten[pixel]
  eudistance = []
  for sample in samples:
    row_sample = sample[1]
    column_sample = sample[0]
    r_sample = array_RGB_copy[row_sample, column_sample, 0]
    g_sample = array_RGB_copy[row_sample, column_sample, 1]
    b_sample = array_RGB_copy[row_sample, column_sample, 2]
    dst = distance.euclidean(np.array((r, g, b)).astype(float), np.array((r_sample, g_sample, b_sample)).astype(float))
    eudistance.append([dst,sample[2]])
    #eudistance.append([dst,sample[2],np.array((r, g, b)).astype(float), np.array((r_sample, g_sample, b_sample)).astype(float)])
  sortdata = sorted(eudistance)
  classe = []
  for i in sortdata[:k]:
      classe.append(i[1])
  cf = max(classe,key=classe.count)  
  
  classification.append(cf)

In [ ]:
## Convert kNN Classification result into array and export
ax = np.array(classification)
axx = ax.reshape(rows,columns)
np.savetxt("final.csv", axx, delimiter=",")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.hist(axx.ravel())


In [ ]:
## Display Classification Image Result and export
plt.imshow(axx,'Accent')
plt.colorbar()
plt.savefig('Classification_R.png',dpi=500)

In [68]:
##Import Verification Label Data
! gdown --id 1CbKp7WiiAKxBlPQPgZKrAuc9lSSsHqv8
Verify = pd.read_csv("Verify_data.csv",header=None).values.tolist()

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CbKp7WiiAKxBlPQPgZKrAuc9lSSsHqv8
To: /content/Verify_data.csv
100% 2.14k/2.14k [00:00<00:00, 3.31MB/s]


In [69]:
##Import Classification array result
! gdown --id 1RDHNeLdGqBEXl9S4ScHlG7dl9f71zAIL
axx = np.array(pd.read_csv("final.csv",header=None))

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1RDHNeLdGqBEXl9S4ScHlG7dl9f71zAIL
To: /content/final.csv
100% 22.1M/22.1M [00:00<00:00, 214MB/s]


In [72]:
# Extract array value from classification array and verification label data

check = []
for ver in Verify:
    row_sample = ver[1]
    column_sample = ver[0]
    #check.append([column_sample,row_sample,axx[row_sample, column_sample]])
    check.append(axx[row_sample, column_sample])

ver = []
for i in range(len(Verify)):
  ver.append(Verify[i][2])

In [76]:
from sklearn.metrics import confusion_matrix
confusion_matrix(check, ver)

array([[34,  2,  0,  0],
       [ 3, 48,  0,  0],
       [13,  0, 49, 16],
       [ 0,  0,  1, 34]])

In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(check,ver)

0.825